In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import re

In [3]:
df = pd.read_csv('result.csv')

In [4]:
df.head(10)

,Unnamed: 0,Reports,ACC No.,BIRADS,Mmg Calcification,Mmg Mass,Disease Name
0,0,Clinical: Routine annual mammog...,28412120-45,BIRADS2,1.0,1.0,alcification|ass
1,1,CLINICAL: Screening~with no presenting sympto...,28408002-138,BIRADS3,1.0,1.0,alcification|ass
2,2,"CLINICAL: palpable mass left 2-3 o'clock,post ...",28413118-47,BIRADS6,1.0,1.0,alcification|ass
3,3,Clinical information: Screening ~ ~BILATERAL ...,28905280-118,BIRADS4B,1.0,NaN,alcification
4,4,CLINICAL: Screening~BILATERAL DIGITAL MAMMOGRA...,28883412-47,BIRADS2,1.0,1.0,alcification|ass
5,5,Clinical information:Screening~BILATERAL DIGIT...,28935614-58,BIRADS2,1.0,NaN,alcification
6,6,~CLINICAL: Follow ...,28854593-90,BIRADS3,1.0,1.0,alcification|ass
7,7,~CLINICAL: Follow ...,28854593-120,BIRADS3,1.0,1.0,alcification|ass
8,8,~CLINICAL: screenin...,22819148-48,BIRADS2,2.0,1.0,alcification|ass
9,9,Clinical information:Case right breast cancer ...,29451428-286,BIRADS2,0.0,0.0,NaN


In [5]:
df.Reports = df.Reports.apply(lambda x: x.replace('~','\n'))

result = [] 
no_pattern = "no.{0,40}calcif|No.{0,40}calcif"
for index,row in df.iterrows():
    #report with impression
    if re.search('Impression|IMPRESSION|IMP|Imp',row[1]):
        imp = re.split('Impression|IMPRESSION|IMP|Imp',row[1])[1]
        detail = re.split('Impression|IMPRESSION|IMP|Imp',row[1])[0]
        impsentence = imp.split('-')
        detailsentence = str(detail).split('.')
        # if there is calcified wording in imp - append result
        if re.search('calcif',imp) : 
            for iteration,sentence in enumerate(impsentence):
                # if this is not the last sentence.
                if iteration < len(impsentence)-1:
                    if re.search(no_pattern,sentence):
                        continue
                    elif re.search('calcif',sentence) :
                        result.append(1.0)
                        break
                # if this is the last sentence in paragraph
                else :
                    if re.search(no_pattern,sentence):
                        result.append(0.0)
                    elif re.search('calcif',sentence) :
                        result.append(1.0)
                    else :
                        result.append(0.0)
        # if there is no calcified wording in imp - devide 2 type of report 
        elif re.search('calcif',detail):
            if re.search('Macrocalcification|Microcalcification',detail):
                detail = detail.replace('\nMicrocalcification','')
                detail = detail.replace('\nMacrocalcification','')
                if re.search('calcif',detail):
                    result.append(1.0)
                else :
                    result.append(0.0)
            else :
                for iteration,sentence in enumerate(detailsentence):                  
                    if iteration < len(detailsentence)-1:
                        if re.search(no_pattern,sentence):
                            continue
                        elif re.search('calcif',sentence) :
                            result.append(1.0)
                            break
                    else :
                        if re.search(no_pattern,sentence):
                             result.append(0.0)
                        elif re.search('calcif',sentence) :
                            result.append(1.0)
                        else :
                            result.append(0.0)
                        
        # in case there is no calcified in IMP and detail at all
        else :
            result.append(0.0)
            
                
        # in cases there is no IMP 
    else:
        if re.search('calcif',row[1]):
            if re.search('Macrocalcification|Microcalcification',row[1]):
                row[1] = row[1].replace('\nMicrocalcification','')
                row[1] = row[1].replace('\nMacrocalcification','')
                if re.search('calcif',row[1]):
                    result.append(1.0)
                else :
                    result.append(0.0)
            else :
                for iteration,sentence in enumerate(row[1].split('.')):                  
                    if iteration < len(row[1].split('.'))-1:
                        if re.search(no_pattern,sentence):
                            continue
                        elif re.search('calcif',sentence) :
                            result.append(1.0)
                            break
                    else :
                        if re.search(no_pattern,sentence):
                             result.append(0.0)
                        elif re.search('calcif',sentence) :
                            result.append(1.0)
                        else :
                            result.append(0.0)
        
        else : 
            result.append(0.0)         
            
            

In [6]:
result = [] 
no_pattern = "no.{0,80}calcif|No.{0,80}calcif"
no_change = "no.{0,20}change|No.{0,20}change"
calcification = 'calcif|calfication|oalcification'
tomo_pattern = 'Tomosynthesis|TOMOSYNTHESIS|tomosynthesis|omogra|OMOGRA'
for index,row in df.iterrows(): 
    if re.search(tomo_pattern,row[1]):
        result.append('Tomosynthesis')
        continue
    if re.search(calcification,row[1]):
        if re.search('\nMacrocalcification|\nMicrocalcification',row[1]):            
            row[1] = row[1].replace('\nMicrocalcification','\n')
            row[1] = row[1].replace('\nMacrocalcification','\n')
        allsentence = re.split('\\.|-|\n:',row[1])
        for iteration,sentence in enumerate(allsentence):                  
            if iteration < len(allsentence)-1:
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            result.append(1.0)
                            break
                    else :
                        continue
                elif re.search(calcification,sentence) :
                    result.append(1.0)
                    break
            else :
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            result.append(1.0)
                    else :
                        result.append(0.0)
                elif re.search(calcification,sentence) :
                    result.append(1.0)
                else :
                    result.append(0.0)
        
    else : 
        result.append(0.0)         
            

In [7]:
# only count for suspicious calcification 
resultcalc = [] 
no_pattern = "no.{0,80}calcif|No.{0,80}calcif|enign.{0,40}calcif"
no_change = "no.{0,20}change|No.{0,20}change"
calcification = 'morphous.{0,20}calcif|eterogeneous.{0,20}calcif|inear.{0,20}calcif|leomorph.{0,20}calcif|egment.{0,20}calcif|calc.{0,20}inside mass|roup.{0,10}(round|puncta){0,20}calcif'
tomo_pattern = 'Tomosynthesis|TOMOSYNTHESIS|tomosynthesis|omogra|OMOGRA'
for index,row in df.iterrows():
    if re.search(tomo_pattern,row[1]):
        resultcalc.append('Tomosynthesis')
        continue
    if re.search(calcification,row[1]):
        if re.search('\nMacrocalcification|\nMicrocalcification',row[1]):            
            row[1] = row[1].replace('\nMicrocalcification','\n')
            row[1] = row[1].replace('\nMacrocalcification','\n')
        allsentence = re.split('\\.|-|\n:',row[1])
        for iteration,sentence in enumerate(allsentence):                  
            if iteration < len(allsentence)-1:
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            resultcalc.append(1.0)
                            break
                            
                    else :
                        continue
                elif re.search(calcification,sentence) :
                    resultcalc.append(1.0)
                    break
            else :
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            resultcalc.append(1.0)
                    else :
                        resultcalc.append(0.0)
                elif re.search(calcification,sentence) :
                    resultcalc.append(1.0)
                else :
                    resultcalc.append(0.0)
        
    else : 
        resultcalc.append(0.0)         
            

In [8]:
print(df.Reports.loc[57])

STEREOTACTIC GUIDED CORE NEEDLE BIOPSY:
PREVIOUS MAMMOGRAMS: 
      - Low suspicion for malignancy of two groups of mild pleomorphic microcalcifications at superficial right upper outer quadrant.
 
PROCEDURE:  Routine stereotactic core needle biopsy was performed under local anesthesia and sterile technique.
      Six core specimens were obtained.
      All of core specimens contain microcalcifications.
      There was some incisional bleeding.
      The study was stopped.
      Specimens were fixed in 10% formalin and sent for pathology.
IMPRESSION: 
      CORE NEEDLE BIOPSY SUCESSFULLY PERFORMED.
      Suggest to appointment with breast surgeon when pathology result complete.



In [30]:
resultmass = [] 
remove_pattern = 'obscure.{0,10}\n.{0,15}(nodule|mass)|obscure.{0,60}(mass|odule)|\nMass|\nSolid mass|(linical|LINICAL|P\.E\.).{0,60}(mass|odule)|uggest breast self examination.*?\n'
no_pattern = 'no.{0,60}mass|No.{0,60}mass|without.{0,60}mass'
no_change_pattern = "no.{0,20}change|No.{0,20}change"
ultrasound_pattern = 'Ultrasound|ULTRASOUND|ultrasound|SONOGRA|Sonogra|sonogra'
mass_pattern = 'mass|nodu'
previous_pattern = 'Previous.{0,20}|previous.{0.20}'
recommendation_pattern = 'Recom|RECOM|Reccom|RECCOM|recom'
tomo_pattern = 'Tomosynthesis|TOMOSYNTHESIS|tomosynthesis|omogra|OMOGRA'
for index,row in df.iterrows():
    if re.search(tomo_pattern,row[1]):
        resultmass.append('Tomosynthesis')
        continue
    # remove false positive pattern 
    if re.search(remove_pattern,row[1]):
        row[1]=re.sub(remove_pattern,'\n',row[1])
    # remove recommendation part and note
    if re.split(recommendation_pattern,row[1]) == 2:
        row[1] = re.split(recommendation_pattern,row[1])[0]
    #find ultrasound in report if in row 1-4 - dont use that report to interpret
    if re.search(ultrasound_pattern,row[1]):
        # if there is ultrasound in first 4 line of report
        if len(re.findall('\n',re.split(ultrasound_pattern,row[1])[0]))<=3 :
            sentencein4line = re.split('\\.|-|\n:',re.split(ultrasound_pattern,row[1])[0])
            # look whether there is 'previous' in last line before splited by ultrasoud or not
            if re.search(previous_pattern,sentencein4line[-1]):
                row[1]= re.split(ultrasound_pattern,row[1])[1]                
            else :
                # if there is no seperate part of MMG and ultrasound - append nan
                if len(re.split(ultrasound_pattern,row[1])) == 2:
                    resultmass.append(np.nan)
                    continue
                # if there seperate part of MMG and ULT , use the second part 
                else :
                    row[1]= re.split(ultrasound_pattern,row[1])[1]  
        else :
            row[1]= re.split(ultrasound_pattern,row[1])[0]
    allsentence = re.split('\\.|-|\n:',row[1])
    for iteration,sentence in enumerate(allsentence): 
        if iteration < len(allsentence)-1:
            if re.search(mass_pattern,sentence):
                if re.search(no_pattern,sentence):
                    if re.search(no_change_pattern,sentence):
                        resultmass.append(1.0)
                        break
                    else :    
                        continue
                else :
                    resultmass.append(1.0)
                    break
        else :
            if re.search(mass_pattern,sentence):
                if re.search(no_pattern,sentence):
                    resultmass.append(0.0)
                else :
                    resultmass.append(1.0)
            else :
                resultmass.append(0.0)

In [27]:
len(resultmass)

15603

In [11]:
Sresultmass = pd.Series(resultmass) 
Sresultmass[Sresultmass.isna()]

222      NaN
619      NaN
2029     NaN
2030     NaN
2032     NaN
        ... 
15451    NaN
15473    NaN
15488    NaN
15566    NaN
15572    NaN
Length: 702, dtype: object

In [12]:
list(enumerate(resultcalc))

[(0, 0.0),
 (1, 0.0),
 (2, 1.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 'Tomosynthesis'),
 (13, 0.0),
 (14, 0.0),
 (15, 1.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 'Tomosynthesis'),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 1.0),
 (52, 'Tomosynthesis'),
 (53, 1.0),
 (54, 0.0),
 (55, 0.0),
 (56, 1.0),
 (57, 1.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0),
 (66, 0.0),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.0),
 (72, 0.0),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 0.0),
 (77, 0.0),
 (78, 0.0),
 (79, 0.0),
 (80, 0.0),
 (

In [13]:
resultmass[250]

1.0

In [14]:
print(df.Reports.loc[250])

Clinical information:Follow up study
BILATERAL DIGITAL MAMMOGRAPHY:
Technique:standard CC, MLO views
Comparison study:to 31 Aug 2018  
Breast density      Heterogeneously dense fibroglandular tissue with could may obscures detection of small masses.
Mass  No significant change over two years of a circumscribed isodense mass at left upper outer quadrant
MicrocalcificationsNot detectable
Macrocalcification  Macrocalcifications in both breasts
Architectural distortionNot detectable
Focal asymmetry/Focal densityNot detectable
Dilated tubular structureNot detectable
Skin thickeningNot detectable
Associated findings:Not detectable
HANDHELD ULTRASOUND:
Breast tissue composition      Heterogeneous background
Cystic massNot detectable
Solid mass  Stable of a 5x4.7x6.5 mm oval hypoechoic nodule without vascularity at left upper outer quadrant (1 o'clock)
Architectural distortionNot detectable
Associated findings-Benign appearing axillary lymph nodes, bilaterally.
   - Two calcific foci at right 

In [15]:
df

,Unnamed: 0,Reports,ACC No.,BIRADS,Mmg Calcification,Mmg Mass,Disease Name
0,0,Clinical: Routine annual mammog...,28412120-45,BIRADS2,1.0,1.0,alcification|ass
1,1,CLINICAL: Screening\nwith no presenting sympt...,28408002-138,BIRADS3,1.0,1.0,alcification|ass
2,2,"CLINICAL: palpable mass left 2-3 o'clock,post ...",28413118-47,BIRADS6,1.0,1.0,alcification|ass
3,3,Clinical information: Screening \n \nBILATERA...,28905280-118,BIRADS4B,1.0,NaN,alcification
4,4,CLINICAL: Screening\nBILATERAL DIGITAL MAMMOGR...,28883412-47,BIRADS2,1.0,1.0,alcification|ass
...,...,...,...,...,...,...,...
15598,15601,"SCREENING BILATERAL MAMMOGRAM CC, MLO VIEWS WI...",34859110-546,BIRADS3,1.0,-1.0,alcification|ass
15599,15602,Bilateral mammogram and ultrasound\nClinical H...,34859565-356,BIRADS2,0.0,0.0,NaN
15600,15603,Bilateral mammogram and ultrasound\nClinical H...,34859674-28,BIRADS3,NaN,NaN,NaN
15601,15604,CK-Ultrasound Breast (P)\n \nClinical :Previou...,34860437-110,BIRADS3,NaN,1.0,ass


In [33]:
df['All_calcification']=result
df['Suspicious_calcification'] = resultcalc
df['Mass'] = resultmass

In [34]:
df.to_csv('GT.csv')

In [18]:
resultmass[250]

1.0

In [19]:
tomo2 = 0
for i in df['Reports'] : 
    if re.search(tomo_pattern,i):
        tomo2 = tomo2+1
print(tomo2)

5014


In [20]:
15603 - 5014

10589

In [21]:
for i in enumerate(resultmass):
    print(i)

(0, 0.0)
(1, 0.0)
(2, 1.0)
(3, 0.0)
(4, 0.0)
(5, 0.0)
(6, 1.0)
(7, 1.0)
(8, 0.0)
(9, 0.0)
(10, 0.0)
(11, 0.0)
(12, 'Tomosynthesis')
(13, 0.0)
(14, 0.0)
(15, 0.0)
(16, 0.0)
(17, 0.0)
(18, 0.0)
(19, 0.0)
(20, 0.0)
(21, 0.0)
(22, 0.0)
(23, 0.0)
(24, 0.0)
(25, 0.0)
(26, 0.0)
(27, 0.0)
(28, 0.0)
(29, 0.0)
(30, 0.0)
(31, 0.0)
(32, 0.0)
(33, 0.0)
(34, 0.0)
(35, 'Tomosynthesis')
(36, 0.0)
(37, 0.0)
(38, 0.0)
(39, 0.0)
(40, 0.0)
(41, 0.0)
(42, 0.0)
(43, 0.0)
(44, 0.0)
(45, 0.0)
(46, 0.0)
(47, 0.0)
(48, 1.0)
(49, 1.0)
(50, 0.0)
(51, 0.0)
(52, 'Tomosynthesis')
(53, 1.0)
(54, 0.0)
(55, 0.0)
(56, 0.0)
(57, 0.0)
(58, 0.0)
(59, 0.0)
(60, 0.0)
(61, 0.0)
(62, 0.0)
(63, 0.0)
(64, 0.0)
(65, 0.0)
(66, 0.0)
(67, 0.0)
(68, 1.0)
(69, 0.0)
(70, 1.0)
(71, 0.0)
(72, 0.0)
(73, 0.0)
(74, 0.0)
(75, 0.0)
(76, 0.0)
(77, 0.0)
(78, 1.0)
(79, 0.0)
(80, 1.0)
(81, 1.0)
(82, 0.0)
(83, 1.0)
(84, 0.0)
(85, 0.0)
(86, 0.0)
(87, 0.0)
(88, 0.0)
(89, 0.0)
(90, 1.0)
(91, 0.0)
(92, 'Tomosynthesis')
(93, 1.0)
(94, 1.0)
(95, 1.0)
(9

(2345, 0.0)
(2346, 0.0)
(2347, 0.0)
(2348, 0.0)
(2349, 0.0)
(2350, 0.0)
(2351, 0.0)
(2352, 0.0)
(2353, 0.0)
(2354, 0.0)
(2355, 1.0)
(2356, 0.0)
(2357, 0.0)
(2358, 1.0)
(2359, 0.0)
(2360, 0.0)
(2361, 0.0)
(2362, 0.0)
(2363, 1.0)
(2364, 0.0)
(2365, 0.0)
(2366, 0.0)
(2367, 0.0)
(2368, 0.0)
(2369, 0.0)
(2370, 1.0)
(2371, 1.0)
(2372, 0.0)
(2373, 1.0)
(2374, 0.0)
(2375, 0.0)
(2376, 0.0)
(2377, 0.0)
(2378, 0.0)
(2379, 0.0)
(2380, 0.0)
(2381, 0.0)
(2382, 1.0)
(2383, 0.0)
(2384, 0.0)
(2385, 0.0)
(2386, 0.0)
(2387, 1.0)
(2388, 0.0)
(2389, 1.0)
(2390, 1.0)
(2391, 0.0)
(2392, 0.0)
(2393, 0.0)
(2394, 0.0)
(2395, 0.0)
(2396, 0.0)
(2397, 0.0)
(2398, 0.0)
(2399, 0.0)
(2400, 0.0)
(2401, 0.0)
(2402, 0.0)
(2403, 0.0)
(2404, 1.0)
(2405, 0.0)
(2406, 0.0)
(2407, 1.0)
(2408, 0.0)
(2409, 0.0)
(2410, 0.0)
(2411, 0.0)
(2412, 0.0)
(2413, 0.0)
(2414, 0.0)
(2415, 0.0)
(2416, 0.0)
(2417, 1.0)
(2418, 0.0)
(2419, 1.0)
(2420, 0.0)
(2421, 0.0)
(2422, 0.0)
(2423, 0.0)
(2424, 0.0)
(2425, 1.0)
(2426, 0.0)
(2427, 0.0)
(242

(4844, 0.0)
(4845, 1.0)
(4846, 0.0)
(4847, 0.0)
(4848, 0.0)
(4849, nan)
(4850, 0.0)
(4851, 0.0)
(4852, 0.0)
(4853, 0.0)
(4854, 0.0)
(4855, 0.0)
(4856, 0.0)
(4857, 0.0)
(4858, 0.0)
(4859, 0.0)
(4860, 0.0)
(4861, 0.0)
(4862, 0.0)
(4863, 0.0)
(4864, 0.0)
(4865, 0.0)
(4866, 0.0)
(4867, 1.0)
(4868, 0.0)
(4869, 0.0)
(4870, 0.0)
(4871, 0.0)
(4872, 0.0)
(4873, 0.0)
(4874, 0.0)
(4875, 0.0)
(4876, 0.0)
(4877, 1.0)
(4878, 0.0)
(4879, 0.0)
(4880, 0.0)
(4881, 0.0)
(4882, 1.0)
(4883, 1.0)
(4884, 0.0)
(4885, 0.0)
(4886, 0.0)
(4887, 1.0)
(4888, 0.0)
(4889, 0.0)
(4890, 0.0)
(4891, 0.0)
(4892, 0.0)
(4893, 0.0)
(4894, 1.0)
(4895, 0.0)
(4896, 0.0)
(4897, 0.0)
(4898, 0.0)
(4899, 0.0)
(4900, 0.0)
(4901, 1.0)
(4902, 0.0)
(4903, 0.0)
(4904, 0.0)
(4905, 1.0)
(4906, 0.0)
(4907, 0.0)
(4908, 0.0)
(4909, 1.0)
(4910, 1.0)
(4911, 0.0)
(4912, 0.0)
(4913, 0.0)
(4914, 0.0)
(4915, 0.0)
(4916, 1.0)
(4917, 1.0)
(4918, 0.0)
(4919, 0.0)
(4920, 0.0)
(4921, 0.0)
(4922, 0.0)
(4923, 0.0)
(4924, 0.0)
(4925, 0.0)
(4926, 0.0)
(492

(7343, 0.0)
(7344, 'Tomosynthesis')
(7345, nan)
(7346, 'Tomosynthesis')
(7347, 'Tomosynthesis')
(7348, 0.0)
(7349, 0.0)
(7350, 'Tomosynthesis')
(7351, 0.0)
(7352, 0.0)
(7353, 0.0)
(7354, 'Tomosynthesis')
(7355, 'Tomosynthesis')
(7356, 0.0)
(7357, 0.0)
(7358, 'Tomosynthesis')
(7359, 'Tomosynthesis')
(7360, 'Tomosynthesis')
(7361, 'Tomosynthesis')
(7362, 'Tomosynthesis')
(7363, 0.0)
(7364, 'Tomosynthesis')
(7365, 'Tomosynthesis')
(7366, 'Tomosynthesis')
(7367, 'Tomosynthesis')
(7368, 'Tomosynthesis')
(7369, 1.0)
(7370, 0.0)
(7371, 'Tomosynthesis')
(7372, 0.0)
(7373, 'Tomosynthesis')
(7374, 'Tomosynthesis')
(7375, 'Tomosynthesis')
(7376, 'Tomosynthesis')
(7377, 0.0)
(7378, 'Tomosynthesis')
(7379, 'Tomosynthesis')
(7380, 0.0)
(7381, 'Tomosynthesis')
(7382, 'Tomosynthesis')
(7383, 'Tomosynthesis')
(7384, 0.0)
(7385, 0.0)
(7386, 'Tomosynthesis')
(7387, 0.0)
(7388, 0.0)
(7389, 0.0)
(7390, 'Tomosynthesis')
(7391, 0.0)
(7392, 0.0)
(7393, 'Tomosynthesis')
(7394, 1.0)
(7395, 0.0)
(7396, 0.0)
(739

(7843, nan)
(7844, 'Tomosynthesis')
(7845, nan)
(7846, nan)
(7847, 0.0)
(7848, 'Tomosynthesis')
(7849, 'Tomosynthesis')
(7850, 0.0)
(7851, 'Tomosynthesis')
(7852, 'Tomosynthesis')
(7853, 0.0)
(7854, nan)
(7855, 'Tomosynthesis')
(7856, 'Tomosynthesis')
(7857, 'Tomosynthesis')
(7858, 'Tomosynthesis')
(7859, 'Tomosynthesis')
(7860, 'Tomosynthesis')
(7861, 0.0)
(7862, 0.0)
(7863, 'Tomosynthesis')
(7864, 'Tomosynthesis')
(7865, nan)
(7866, 'Tomosynthesis')
(7867, 'Tomosynthesis')
(7868, 'Tomosynthesis')
(7869, nan)
(7870, 'Tomosynthesis')
(7871, 'Tomosynthesis')
(7872, nan)
(7873, 'Tomosynthesis')
(7874, 'Tomosynthesis')
(7875, 'Tomosynthesis')
(7876, nan)
(7877, nan)
(7878, 'Tomosynthesis')
(7879, 'Tomosynthesis')
(7880, 'Tomosynthesis')
(7881, 'Tomosynthesis')
(7882, 'Tomosynthesis')
(7883, 0.0)
(7884, 'Tomosynthesis')
(7885, 'Tomosynthesis')
(7886, 'Tomosynthesis')
(7887, 'Tomosynthesis')
(7888, nan)
(7889, 'Tomosynthesis')
(7890, 'Tomosynthesis')
(7891, 'Tomosynthesis')
(7892, 'Tomosynt

...

(10342, 'Tomosynthesis')
(10343, 'Tomosynthesis')
(10344, 0.0)
(10345, 0.0)
(10346, 1.0)
(10347, 'Tomosynthesis')
(10348, 0.0)
(10349, 0.0)
(10350, 'Tomosynthesis')
(10351, 'Tomosynthesis')
(10352, 'Tomosynthesis')
(10353, 'Tomosynthesis')
(10354, 'Tomosynthesis')
(10355, 'Tomosynthesis')
(10356, 'Tomosynthesis')
(10357, 0.0)
(10358, 'Tomosynthesis')
(10359, 0.0)
(10360, 'Tomosynthesis')
(10361, 'Tomosynthesis')
(10362, 'Tomosynthesis')
(10363, 'Tomosynthesis')
(10364, 'Tomosynthesis')
(10365, 'Tomosynthesis')
(10366, 0.0)
(10367, 0.0)
(10368, 'Tomosynthesis')
(10369, 0.0)
(10370, 'Tomosynthesis')
(10371, 'Tomosynthesis')
(10372, 'Tomosynthesis')
(10373, 0.0)
(10374, 'Tomosynthesis')
(10375, 0.0)
(10376, 0.0)
(10377, 'Tomosynthesis')
(10378, 0.0)
(10379, 'Tomosynthesis')
(10380, 'Tomosynthesis')
(10381, 0.0)
(10382, 'Tomosynthesis')
(10383, 0.0)
(10384, 'Tomosynthesis')
(10385, 'Tomosynthesis')
(10386, 'Tomosynthesis')
(10387, 'Tomosynthesis')
(10388, 'Tomosynthesis')
(10389, 'Tomosynt

(12341, 'Tomosynthesis')
(12342, 'Tomosynthesis')
(12343, 'Tomosynthesis')
(12344, 'Tomosynthesis')
(12345, 'Tomosynthesis')
(12346, 'Tomosynthesis')
(12347, nan)
(12348, 1.0)
(12349, 'Tomosynthesis')
(12350, nan)
(12351, 'Tomosynthesis')
(12352, 'Tomosynthesis')
(12353, 'Tomosynthesis')
(12354, nan)
(12355, 'Tomosynthesis')
(12356, 'Tomosynthesis')
(12357, 'Tomosynthesis')
(12358, 'Tomosynthesis')
(12359, 'Tomosynthesis')
(12360, nan)
(12361, 'Tomosynthesis')
(12362, 0.0)
(12363, 0.0)
(12364, 0.0)
(12365, 'Tomosynthesis')
(12366, 1.0)
(12367, 'Tomosynthesis')
(12368, 'Tomosynthesis')
(12369, 1.0)
(12370, 'Tomosynthesis')
(12371, 'Tomosynthesis')
(12372, 'Tomosynthesis')
(12373, nan)
(12374, 0.0)
(12375, nan)
(12376, 'Tomosynthesis')
(12377, 0.0)
(12378, 'Tomosynthesis')
(12379, 'Tomosynthesis')
(12380, 0.0)
(12381, 1.0)
(12382, 'Tomosynthesis')
(12383, 'Tomosynthesis')
(12384, 'Tomosynthesis')
(12385, 'Tomosynthesis')
(12386, 'Tomosynthesis')
(12387, 'Tomosynthesis')
(12388, 'Tomosynt

(12841, 0.0)
(12842, 'Tomosynthesis')
(12843, 0.0)
(12844, 'Tomosynthesis')
(12845, 'Tomosynthesis')
(12846, nan)
(12847, 'Tomosynthesis')
(12848, 'Tomosynthesis')
(12849, 'Tomosynthesis')
(12850, 0.0)
(12851, 'Tomosynthesis')
(12852, 0.0)
(12853, 0.0)
(12854, 0.0)
(12855, 0.0)
(12856, 0.0)
(12857, 0.0)
(12858, 'Tomosynthesis')
(12859, 0.0)
(12860, 'Tomosynthesis')
(12861, 0.0)
(12862, 0.0)
(12863, 0.0)
(12864, 0.0)
(12865, 0.0)
(12866, 'Tomosynthesis')
(12867, 'Tomosynthesis')
(12868, 0.0)
(12869, 0.0)
(12870, 'Tomosynthesis')
(12871, 0.0)
(12872, 0.0)
(12873, 'Tomosynthesis')
(12874, 0.0)
(12875, 0.0)
(12876, 0.0)
(12877, 0.0)
(12878, 0.0)
(12879, 'Tomosynthesis')
(12880, 'Tomosynthesis')
(12881, 1.0)
(12882, 'Tomosynthesis')
(12883, 'Tomosynthesis')
(12884, 0.0)
(12885, 0.0)
(12886, 0.0)
(12887, 0.0)
(12888, 0.0)
(12889, 0.0)
(12890, nan)
(12891, 'Tomosynthesis')
(12892, 'Tomosynthesis')
(12893, 'Tomosynthesis')
(12894, 'Tomosynthesis')
(12895, 'Tomosynthesis')
(12896, 'Tomosynthesi

(14840, 'Tomosynthesis')
(14841, 'Tomosynthesis')
(14842, 0.0)
(14843, 'Tomosynthesis')
(14844, 'Tomosynthesis')
(14845, 'Tomosynthesis')
(14846, 0.0)
(14847, 0.0)
(14848, 'Tomosynthesis')
(14849, 'Tomosynthesis')
(14850, 0.0)
(14851, 0.0)
(14852, 'Tomosynthesis')
(14853, 'Tomosynthesis')
(14854, 'Tomosynthesis')
(14855, 'Tomosynthesis')
(14856, 0.0)
(14857, 'Tomosynthesis')
(14858, 0.0)
(14859, 'Tomosynthesis')
(14860, 'Tomosynthesis')
(14861, 'Tomosynthesis')
(14862, 0.0)
(14863, 'Tomosynthesis')
(14864, 0.0)
(14865, 0.0)
(14866, 'Tomosynthesis')
(14867, 'Tomosynthesis')
(14868, 'Tomosynthesis')
(14869, nan)
(14870, 'Tomosynthesis')
(14871, 'Tomosynthesis')
(14872, 'Tomosynthesis')
(14873, 'Tomosynthesis')
(14874, 0.0)
(14875, 'Tomosynthesis')
(14876, 'Tomosynthesis')
(14877, 0.0)
(14878, 'Tomosynthesis')
(14879, 'Tomosynthesis')
(14880, 'Tomosynthesis')
(14881, 0.0)
(14882, 'Tomosynthesis')
(14883, 'Tomosynthesis')
(14884, 'Tomosynthesis')
(14885, 'Tomosynthesis')
(14886, nan)
(1488

(15340, 0.0)
(15341, 'Tomosynthesis')
(15342, 0.0)
(15343, 0.0)
(15344, nan)
(15345, 'Tomosynthesis')
(15346, 'Tomosynthesis')
(15347, 'Tomosynthesis')
(15348, 'Tomosynthesis')
(15349, 'Tomosynthesis')
(15350, 'Tomosynthesis')
(15351, 0.0)
(15352, 0.0)
(15353, 0.0)
(15354, 'Tomosynthesis')
(15355, 0.0)
(15356, nan)
(15357, nan)
(15358, 'Tomosynthesis')
(15359, nan)
(15360, 'Tomosynthesis')
(15361, 'Tomosynthesis')
(15362, nan)
(15363, 0.0)
(15364, 'Tomosynthesis')
(15365, 0.0)
(15366, 0.0)
(15367, nan)
(15368, 'Tomosynthesis')
(15369, 0.0)
(15370, 'Tomosynthesis')
(15371, 0.0)
(15372, 0.0)
(15373, 0.0)
(15374, 0.0)
(15375, 0.0)
(15376, 0.0)
(15377, 1.0)
(15378, 1.0)
(15379, 'Tomosynthesis')
(15380, 'Tomosynthesis')
(15381, 0.0)
(15382, 'Tomosynthesis')
(15383, 'Tomosynthesis')
(15384, 0.0)
(15385, 0.0)
(15386, 'Tomosynthesis')
(15387, 0.0)
(15388, 0.0)
(15389, 'Tomosynthesis')
(15390, 1.0)
(15391, 'Tomosynthesis')
(15392, 'Tomosynthesis')
(15393, 0.0)
(15394, 0.0)
(15395, 0.0)
(15396, 